Reads several SQL views/tables via PyODBC, assembles them into a data frame, then filters them according to some criteria and finally matching the items by bar code in preparation for a database migration (in order to avoid duplicate SKUs).

In [1]:
%%time

import warnings

warnings.filterwarnings('ignore')

from functools import reduce

import numpy as np

import pyodbc

cnxn = pyodbc.connect(driver = '{ODBC Driver 17 for SQL Server}', server = 'tcp:10.3.6.50', database = 'reports', trusted_connection = 'yes')

cursor = cnxn.cursor()

import pandas as pd

#####
#HRM items
#####

query = '''

SELECT *

FROM HRM_items

'''

dfHRM = pd.read_sql(query, cnxn)

dfHRM.drop(['ST', 'UPC_Date_Last_Used'], axis = 1, inplace = True)

dfHRM.rename(columns={'Item_Number_SKU': 'ITEMID', 'Item_Mfg_Part_Number': 'ExternalID', 'Item_Description': 'Description', 'Curent_Vendor_Name': 'VendorName', 
                      'Current_Manufacturer_Code': 'MfgID', 'Current_Manufacturer_Name': 'MfgName','UPC_Code': 'BarCode', 'Vendor_Code': 'VendorID', 'Current_Department_Code': 'DeptID', 
                      'Current_Department_Name': 'DeptName', 'Current_Class_Code': 'ClassID', 'Current_Class_Name': 'ClassName', 'Current_Fineline_Code': 'FinelineID', 
                      'Current_Fineline_Name': 'FinelineName'}, inplace=True)


#####
#AX items
#####

query = '''

SELECT *

FROM SD_items_by_vendor_UPC2

'''

dfLHA = pd.read_sql(query, cnxn)


#####
#item groups
#####

query = '''

SELECT *

FROM SD_Item_Groups

'''

dfItemGroups = pd.read_sql(query, cnxn)


#####
#purchase pricing
#####

query = '''

SELECT *

FROM SD_Current_Purchase_Price

'''

dfPurchasePrice = pd.read_sql(query, cnxn)


#####
#sales by item
#####

query = '''

SELECT *

FROM SD_Sales_By_Item

'''

dfSalesByItem = pd.read_sql(query, cnxn)


#####
#web price
#####

query = '''

SELECT *

FROM SD_web_pricing

'''

dfWebPrice = pd.read_sql(query, cnxn)


#####
#retail price
#####

query = '''

SELECT *

FROM SD_Current_Retail_Price

'''

dfRetailPrice = pd.read_sql(query, cnxn)

#####
#categories
#####

query = '''

SELECT *

FROM SD_AX_Item_Categories

'''

dfCategories = pd.read_sql(query, cnxn)

dfLHA.drop(dfLHA.columns[[6,7,8,9,10,13]], axis = 1, inplace = True)

dfLHA.rename(columns={'NAMEALIAS': 'Description_LHA', 'PrimaryVendorId': 'PrimaryVendorID_LHA', 'Vendor Name': 'PrimaryVendorName_LHA',
                            'ExternalID': 'ExternalID_LHA', 'BarCode':'BarCode_LHA'}, inplace=True)

# make sure all vendor IDs have an uppercase V
dfPurchasePrice['VENDORID'] = dfPurchasePrice['VENDORID'].str.upper()

dfPurchasePrice.sort_values(by=['ITEMID','VENDORID','FROMDATE','AMOUNT'], inplace=True, ascending=[True,True,False,True])

dfPurchasePrice = dfPurchasePrice.drop_duplicates(subset = ['ITEMID', 'VENDORID'],keep = 'first').reset_index(drop = True)

dfPurchasePrice.drop(dfPurchasePrice.columns[[4]], axis = 1, inplace = True)

dfPurchasePrice.rename(columns={'AMOUNT': 'PurchasePrice_LHA', 'VENDORID': 'VendorID_LHA', 'VendorName': 'VendorName_LHA'}, inplace=True)


dfSalesByItem.drop(dfSalesByItem.columns[[1,2,3,4]], axis = 1, inplace = True)

dfSalesByItem.rename(columns={'Total Qty': 'TotalSalesUnits_LHA', 'Total Sales': 'TotalSales_LHA'}, inplace=True)

dfSalesByItem_Qty = dfSalesByItem.pivot(index='ITEMID', columns='Year', values='TotalSalesUnits_LHA')

dfSalesByItem_Sales = dfSalesByItem.pivot(index='ITEMID', columns='Year', values='TotalSales_LHA')

dfSalesByItem.drop_duplicates(subset ="ITEMID", keep = False, inplace = True)


# compile the list of dataframes you want to merge
data_frames = [dfSalesByItem, dfSalesByItem_Qty, dfSalesByItem_Sales]

dfSalesByItem_merged = reduce(lambda left,right: pd.merge(left,right,on=['ITEMID'], how='outer'), data_frames).fillna('')

dfSalesByItem_merged.rename(columns={'2020_x': '2020_Qty', '2021_x': '2021_Qty', '2022_x': '2022_Qty', 
                                     '2020_y': '2020_Sales', '2021_y': '2021_Sales', '2022_y': '2022_Sales'}, inplace=True)


dfWebPrice.drop(dfWebPrice.columns[[1,2,3,4,6,7]], axis = 1, inplace = True)

dfWebPrice.rename(columns={'WEBPRICE': 'WebPrice_LHA'}, inplace=True)


dfRetailPrice.drop(dfRetailPrice.columns[[2,3]], axis = 1, inplace = True)

dfRetailPrice.rename(columns={'PRICE': 'BaseRetailPrice_LHA'}, inplace=True)


dfCategories.rename(columns={'Category': 'Category_LHA', 'SubCategory': 'SubCategory_LHA', 'SubSubCategory': 'SubSubCategory_LHA', 'SubSubSubCategory': 'SubSubSubCategory_LHA'}, inplace=True)


# compile the list of dataframes you want to merge
data_frames = [dfLHA, dfItemGroups, dfCategories, dfPurchasePrice, dfSalesByItem_merged, dfWebPrice, dfRetailPrice]

df_merged = reduce(lambda left,right: pd.merge(left,right,on=['ITEMID'], how='outer'), data_frames).fillna('')

df_merged.rename(columns={'ITEMID': 'ITEMID_LHA', 'InventoryAvailable': 'InventoryAvailable_LHA', 'ITEMGROUPID': 'Dept_LHA' }, inplace=True)

df_merged = df_merged.reindex(columns=['ITEMID_LHA', 'Description_LHA', 'PrimaryVendorID_LHA','VendorID_LHA','PrimaryVendorName_LHA', 
                                       'VendorName_LHA','ExternalID_LHA', 'BarCode_LHA', 'Discontinued', 'Published', 'SyncCA', 'InventoryAvailable_LHA', 
                                       'Dept_LHA', 'Category_LHA', 'SubCategory_LHA', 'SubSubCategory_LHA', 'SubSubSubCategory_LHA', 
                                       'PurchasePrice_LHA', 'BaseRetailPrice_LHA', 'WebPrice_LHA', '2020_Qty', '2021_Qty', '2022_Qty', 
                                       '2020_Sales', '2021_Sales', '2022_Sales'])


df_merged = df_merged.assign(In_dfHRM = df_merged.BarCode_LHA.isin(dfHRM.BarCode).astype(int))

df_merged = df_merged.merge(dfHRM, left_on=['BarCode_LHA'], right_on=['BarCode'], suffixes=("", "_right"))

df_merged.rename(columns={'In_dfHRM': 'Match'}, inplace=True)

df_merged['2020_Qty'] = pd.to_numeric(df_merged['2020_Qty'], downcast='integer')
df_merged['2021_Qty'] = pd.to_numeric(df_merged['2021_Qty'], downcast='integer')
df_merged['2022_Qty'] = pd.to_numeric(df_merged['2022_Qty'], downcast='integer')
df_merged['InventoryAvailable_LHA'] = pd.to_numeric(df_merged['InventoryAvailable_LHA'], downcast='integer')

# df_merged.to_excel('2022_07_08 HRM LHA BarCode Match FULL.xlsx', index=False, header=True)

print('Ready to filter.')

Ready to filter.
CPU times: total: 7.75 s
Wall time: 37.5 s


In [ ]:
%%time

#all matches that are not discontinued
#where discontinued is not empty, but they have inventory
#or where discontinued is other, but there were sales
df_merged = (df_merged[
                    (df_merged['Match'] == 1) & 
                    (
                        (df_merged['Discontinued'] == '') | 
                        
                         ((df_merged['Discontinued'] == 'Other') & 
                              
                              (((df_merged['2020_Qty'] > 0) | (df_merged['2021_Qty'] > 0) | (df_merged['2022_Qty'] > 0)) |
                               # ((dfLHA_merge['2020_Qty'].isnull().any() == False) | (dfLHA_merge['2021_Qty'].isnull().any() == False) | (dfLHA_merge['2022_Qty'].isnull().any() == False)) |
                               # ((dfLHA_merge['InventoryAvailable_LHA'] > 0) | (dfLHA_merge['InventoryAvailable_LHA'].isnull().any() == False))
                               (df_merged['InventoryAvailable_LHA'] > 0)
                               
                              )
                         ) |
                        
                        ((df_merged['Discontinued'] == 'Vendor Discon') & 
                              
                              (
                               (df_merged['InventoryAvailable_LHA'] > 0)
                               
                              )
                         )
                     )
                ])

df_merged.sort_values(by=['ITEMID_LHA','VendorID_LHA'], inplace=True, ascending=[True,True], ignore_index=True)

# number of rows that are duplicates based on Match

# all rows in which Match is 1
seriesObj = df_merged.apply(lambda x: True if x['Match'] > 0 else False , axis=1)

# count of all Trues in the series
numOfRows = len(seriesObj[seriesObj == True].index)
print('Number of all rows in dataframe matching by bar code: ', numOfRows)

# count of unique SKUs in the dataframe
uniqueMatches = df_merged['ITEMID_LHA'].nunique()

print('Number of unique rows in dataframe matching by bar code: ', uniqueMatches)

numOfRows = len(dfLHA.index)
print('Total number of rows in dataframe: ' , numOfRows)

df_merged.drop('Match', axis = 1, inplace = True)

df_merged.to_excel('2022_09_09 HRM LHA BarCode Match.xlsx', index=False, header=True)

df_merged.head()

In [3]:
df_merged.to_excel('2022_09_09 HRM LHA BarCode Match.xlsx', index=False, header=True)

In [ ]:
df_CategoryComparison = df_merged

df_CategoryComparison.drop_duplicates(subset ='ITEMID_LHA', keep = False, inplace = True)

df_CategoryComparison.reset_index(drop=True, inplace=True)

df_CategoryComparison['Dept_LHA'] = df_CategoryComparison['Dept_LHA'].str.upper()

df_CategoryComparison['ClassID'] = df_CategoryComparison['ClassID'].str.upper()

df_CategoryComparison['FinelineID'] = df_CategoryComparison['FinelineID'].str.upper()

##############
# Departments#
##############
df_CategoryComparison['DepartmentPairings'] = list(zip(df_CategoryComparison.Dept_LHA, df_CategoryComparison.DeptID))

# Get frequency count of values in column 'CategoryPairings'
frequency = df_CategoryComparison['DepartmentPairings'].value_counts()

frequency.to_frame(name='DepartmentPairings').to_excel('2022_07_11 Department Pairings.xlsx', sheet_name='Dept')

#############
# Categories#
#############
df_CategoryComparison['CategoryPairings'] = list(zip(df_CategoryComparison.Category_LHA, df_CategoryComparison.ClassID))

# Get frequency count of values in column 'CategoryPairings'
frequency = df_CategoryComparison['CategoryPairings'].value_counts()

frequency.to_frame(name='CategoryPairings').to_excel('2022_07_11 Category Pairings.xlsx', sheet_name='Categories')

################
# Subcategories#
################
df_CategoryComparison['SubCategoryPairings'] = list(zip(df_CategoryComparison.SubCategory_LHA, df_CategoryComparison.FinelineID))

# Get frequency count of values in column 'CategoryPairings'
frequency = df_CategoryComparison['SubCategoryPairings'].value_counts()

frequency.to_frame(name='SubCategoryPairings').to_excel('2022_07_11 SubCategory Pairings.xlsx', sheet_name='SubCategories')

df_CategoryComparison.head()